In [1]:
import sys
sys.path.append("../base")

import boilerplate
from pyrtl import *

In [2]:
reset_working_block()

pc = Register(name='pc', bitwidth=4)
sp = Register(name='sp', bitwidth=4)
mem = MemBlock(name='mem', bitwidth=4, addrwidth=4)
rom = RomBlock(name='rom', bitwidth=5, addrwidth=4, romdata=[0x15, 0x18, 0, 0xf],
               pad_with_zeros=True)  # needed for C compilation

out = Output(name='out', bitwidth=4)

In [3]:
instr = rom[pc]
out <<= mem[sp]

with conditional_assignment:
    with instr[4] == 1:   # PUSH
        mem[sp] |= instr[0:4]
        sp.next |= sp + 1
        pc.next |= pc + 1
    with instr == 0:      # POP
        sp.next |= sp - 1
        pc.next |= pc + 1

##### Debug ports -- _so that we see some feedback during simulation_

In [4]:
d_sp = Output(name='d_sp', bitwidth=4)
d_pc = Output(name='d_pc', bitwidth=4)

d_sp <<= sp
d_pc <<= pc

#### Display adapter

In [5]:
vid_y = Output(name='vid_y', bitwidth=8)
vid_out = Output(name='vid_out', bitwidth=256)

vid_scan = Register(name='vid_scan', bitwidth=5)
vid_scan.next <<= vid_scan + 1
vid_y <<= 18 + vid_scan
vid_out <<= 0xff0f0ff0000

### Simulation in C

In [6]:
import os
from simulate import CCompiledSimulation

In [7]:
CCompiledSimulation(out_dir="obj")

In [8]:
os.system("make")

gcc -O3 -Iobj -c -o obj/csim.o obj/csim.c
g++ -O3 -Iobj -std=c++11 -DNO_DEBUG_MEM -c -o obj/csim_main.o simulate/csim_main.cpp
g++ -o bin/csim obj/csim.o obj/csim_main.o


0

In [9]:
os.environ['DEBUG_CPU'] = '1'
os.environ['MAX_CYCLES'] = '20'
os.system('bin/csim')

0000 0000 
20|0000000000000000111111110000111100001111111100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
0001 0001 
21|0000000000000000111111110000111100001111111100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
0002 0002 
22|0000000000000000111111110000111100001111111100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
0003 0001 
23|00000000000000001111111100001111000011111111000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

0

2e|0000000000000000111111110000111100001111111100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
0003 0001 
2f|0000000000000000111111110000111100001111111100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
0003 0001 
30|0000000000000000111111110000111100001111111100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
0003 0001 
31|0000000000000000111111110000111100001111111100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000